In [1]:
from os import listdir
import pandas as pd
import numpy as np
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


data_path = '../../data/processed/jiza'
full_path = data_path+'/'+'10 глупых вопрос РЕСТАВРАТОРУ-qO-Qd-QUoUI.txt'
text = pd.read_csv(full_path).columns[0]

filler_words = 'да,точка,это самое,типо того,таки,собственно говоря,прямо,прикол,практически,походу,на самом деле,конкретно,как-то так,как сказать,как его,итак,знаешь,да ладно,все такое,в самом деле,в натуре,вот,эм,ну,вообще-то,так сказать,кстати,в общем,просто,ааа,мм,стало быть'
filler_words = filler_words.split(',')

##### Функция анализа слов-паразитов для MVP

In [59]:
def filler_word_analyse(text, filler_words):
    
    filler_cnt_dict = {}
    
    # Для каждого слова-паразита считаем кол-во вхождений
    for filler in filler_words:
        filler_cnt = text.count(' '+filler+' ')
        if filler_cnt>0:
            filler_cnt_dict[filler] = filler_cnt
    
    # Сортируем от максимального кол-ва вхождений к минимальному
    filler_cnt_dict = {k: v for k, v in sorted(filler_cnt_dict.items(), key=lambda item: -item[1])}
    
    # Общее количество слов
    all_words_cnt = text.count(' ')+1
    
    # Количество слов-паразитов
    all_fillers_cnt = sum(filler_cnt_dict.values())
    
    # Доля филеров в процентах
    fillers_pct = 100*all_fillers_cnt/all_words_cnt
    
    all_words = 'Всего слов - ' + str(all_words_cnt) + ' \n \n'
    filler_words_pct = 'Доля слов паразитов - ' + str(np.round(fillers_pct,1)) +'% \n \n'
    frequent_fillers = 'Встречаемость слов-паразитов: \n'
    
    for filler in filler_cnt_dict:
        filler_frequency = filler + ' - ' + str(filler_cnt_dict[filler]) + '\n'
        frequent_fillers = frequent_fillers + filler_frequency
        
    if all_words==0:
        text_for_user = 'В записи не обнаружено ни одного слова, предлагаем сделать еще одну попытку. Говорите, не стесняйтесь)'
    elif all_fillers_cnt==0:
        text_for_user = 'В записи не обнаружено ни одного мусорного слова. Так держать. Расскажите еще что-нибудь) \n \n' + all_words
    elif fillers_pct<=1: 
        conclusion = 'В вашей речи немного мусорных слов, но кое-что мы нашли. \n \n'
        text_for_user = conclusion + all_words + filler_words_pct + frequent_fillers
    elif 1<fillers_pct<=3: 
        conclusion = 'Слов-паразитов в вашей речи среднее количество, предлагаем еще потренироваться говорить без них. Продолжайте! \n \n'
        text_for_user = conclusion + all_words + filler_words_pct + frequent_fillers
    elif fillers_pct>3: 
        conclusion = 'Слов-паразитов в вашей речи достаточно много, предлагаем потренироваться говорить без них. Продолжайте! \n \n'
        text_for_user = conclusion + all_words + filler_words_pct + frequent_fillers
    
    print(text_for_user)
    
    return text_for_user

In [57]:
text_for_user = filler_word_analyse(text,filler_words)

Слов-паразитов в вашей речи среднее количество, предлагаем еще потренироваться говорить без них. Продолжайте! 
 
Всего слов - 2455 
 
Доля слов паразитов - 1.4% 
 
Встречаемость слов-паразитов: 
вот - 13
да - 6
ну - 5
просто - 5
на самом деле - 4
прямо - 1



##### Статистика-аналитика слов паразитов в выборке

In [20]:
all_files = listdir(data_path)
all_text = ''
for file in all_files:
    add_text = pd.read_csv(data_path+'/'+file).columns[0]
    all_text = all_text + add_text

In [30]:
filler_cnt_dict = {}
    
# Для каждого слова-паразита считаем кол-во вхождений
for filler in filler_words:
    filler_cnt = all_text.count(' '+filler+' ')
    if filler_cnt>0:
        filler_cnt_dict[filler] = filler_cnt
        
# Сортируем от максимального кол-ва вхождений к минимальному
filler_cnt_dict = {k: v for k, v in sorted(filler_cnt_dict.items(), key=lambda item: -item[1])}

In [34]:
# Общее количество слов
all_words_cnt = all_text.count(' ')+1

# Количество слов-паразитов
all_fillers_cnt = sum(filler_cnt_dict.values())

# Доля филеров в процентах
fillers_pct = str(np.round(100*all_fillers_cnt/all_words_cnt,1))

In [35]:
fillers_pct

'2.4'

In [45]:
all_words = 'Всего слов - ' + str(all_words_cnt) + ' \n \n'
filler_pct = 'Доля слов паразитов - ' + fillers_pct +'% \n \n'
frequent_fillers = 'Встречаемость слов-паразитов: \n'

for filler in filler_cnt_dict:
    filler_word_pct = np.round(100*filler_cnt_dict[filler]/all_words_cnt,1)
    filler_frequency = filler + ' - ' + str(filler_word_pct) + '%\n'
    frequent_fillers = frequent_fillers + filler_frequency

analytics = all_words + filler_pct + frequent_fillers

In [46]:
print(analytics)

Всего слов - 209717 
 
Доля слов паразитов - 2.4% 
 
Встречаемость слов-паразитов: 
вот - 0.8%
да - 0.5%
просто - 0.4%
ну - 0.3%
на самом деле - 0.2%
в общем - 0.0%
кстати - 0.0%
прямо - 0.0%
практически - 0.0%
таки - 0.0%
знаешь - 0.0%
так сказать - 0.0%
вообще-то - 0.0%
конкретно - 0.0%
собственно говоря - 0.0%
это самое - 0.0%
точка - 0.0%
как-то так - 0.0%
да ладно - 0.0%
как сказать - 0.0%
как его - 0.0%
итак - 0.0%
все такое - 0.0%
эм - 0.0%

